In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [2]:
print ("Part 1- Toronto Neighborhod Dataframe")

Part 1- Toronto Neighborhod Dataframe


In [3]:
from bs4 import BeautifulSoup #library to handle data scrapping
import requests
import urllib.request #library to get data from url

In [4]:
#get data from website
Source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [5]:
#parse data as text file
canada_data = BeautifulSoup(Source.text, "html.parser")

In [6]:
#checks for table
table = canada_data.find('table',{'class':'wikitable sortable'})

In [7]:
#gather and assign tabular data to rows
table_rows = table.find_all('tr')


In [8]:
data = []
for row in table_rows:
        data.append([t.text.strip() for t in row.find_all('td')])
    
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])

In [9]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [10]:
#df.drop(df[df['PostalCode']=="None"].index,axis=0, inplace=True)
df = df.drop([0])

In [11]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 3 to 179
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
#df = df.reset_index()
#df.head(5)

In [12]:
df.shape

(103, 3)

In [13]:
print ("Part 2- Geographical cordinate")


Part 2- Geographical cordinate


In [14]:
import csv
#import os
import types
from botocore.client import Config
import ibm_boto3


In [16]:
#pwd

'/home/dsxuser/work'

In [15]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
#Merge both tables
df_torontoNeighborhoods = pd.merge(df, df_geocoord, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove one "Postal Code" column
df_torontoNeighborhoods.drop("Postal Code", axis=1, inplace=True)
df_torontoNeighborhoods.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [17]:
print ("Part 3: Explore and cluster neighborhoods")

Part 3: Explore and cluster neighborhoods


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_torontoNeighborhoods['Borough'].unique()),
        df_torontoNeighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [21]:
#get the latitude and longitude values of

address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [22]:
# create map of Toronto using latitude and longitude values. And add markers
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto
 

In [23]:
for lat, lng, borough, neighborhood in zip(
        df_torontoNeighborhoods['Latitude'], 
        df_torontoNeighborhoods['Longitude'], 
        df_torontoNeighborhoods['Borough'], 
        df_torontoNeighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto


In [24]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'YY0FFSMVXFEN4J2JVTBP4VKH0HBDCHL2JCUIFGQZ5OJ1HXAR' # your Foursquare ID
CLIENT_SECRET = 'Y0RM40VZV0YZTK1WYNWCCHTGX5RZSHJTR13MAYQGZIBYDNLG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YY0FFSMVXFEN4J2JVTBP4VKH0HBDCHL2JCUIFGQZ5OJ1HXAR
CLIENT_SECRET:Y0RM40VZV0YZTK1WYNWCCHTGX5RZSHJTR13MAYQGZIBYDNLG


In [25]:
#Explore the first neighborhood in the dataframe
neighbourhood_name = df_torontoNeighborhoods.loc[0, 'Neighbourhood']
print(f"The first neighbourhood's name is '{neighbourhood_name}'.")

The first neighbourhood's name is 'Parkwoods'.


In [26]:
#Get the neighborhood's latitude and longitude values

neighbourhood_latitude = df_torontoNeighborhoods.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = df_torontoNeighborhoods.loc[0, 'Longitude'] # neighbourhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [28]:
#get the top 50 venues that are in The Parkwoods within a radius of 500 meters.

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [29]:
#Function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Sun Life,Construction & Landscaping,43.754760,-79.332783
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [ ]:
#k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_denc_grouped_clustering = toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 